In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
full_df = pd.read_csv("data/cardinals_starters.csv")
pd.options.display.max_columns=None
display(full_df)

,pitcher_id,pitcher_name,pitcher_hand,count_balls,count_strikes,outs,pitch_number,pitch_type,pitch_result,pitch_result_subcat,atbat_result,speed_start,speed_end,movement_horizontal,movement_vertical,pitch_spin_rate,pitch_location_x,pitch_location_y,break_sharpness,break_amount,break_location,nasty,batter_id,batter_name,batter_stance,score_pitcher,score_batter,atbat_id,inning,inning_top,on_1b,on_2b,on_3b,game_id,home_team,away_team,stadium,pitch_location_category,previous_pitch_type,previous_pitch_location
0,593372,Carlos Martinez,R,0,0,1,1,Two-Seamer,Ball,Ball,Single,91.2,83.5,7.480000,9.940000,2423.853,-1.600000,1.980000,-33.0,4.3,23.8,40.0,518792,Jason Heyward,L,0,0,2016015415,1,True,False,False,False,201600206,sln,chn,Busch Stadium,left,none,none
1,593372,Carlos Martinez,R,1,0,1,2,Four-Seamer,Ball,Ball,Single,90.8,83.1,5.240000,10.550000,2284.681,-0.010000,1.200000,-25.1,3.9,23.8,21.0,518792,Jason Heyward,L,0,0,2016015415,1,True,False,False,False,201600206,sln,chn,Busch Stadium,low,Two-Seamer,left
2,593372,Carlos Martinez,R,2,0,1,3,Four-Seamer,Ball,Ball,Single,91.1,83.8,5.330000,11.020000,2400.114,1.210000,1.520000,-30.2,3.7,23.8,67.0,518792,Jason Heyward,L,0,0,2016015415,1,True,False,False,False,201600206,sln,chn,Busch Stadium,low and right,Four-Seamer,low
3,593372,Carlos Martinez,R,3,0,1,4,Four-Seamer,Ball,Ball,Single,90.6,83.7,4.040000,8.380000,1825.515,0.990000,2.590000,-18.2,4.3,23.8,52.0,518792,Jason Heyward,L,0,0,2016015415,1,True,False,False,False,201600206,sln,chn,Busch Stadium,middle right,Four-Seamer,low and right
4,593372,Carlos Martinez,R,0,0,1,1,Slider,Swinging Strike,Foul Ball,Flyout,86.3,79.4,4.010000,0.360000,740.567,0.510000,1.730000,-12.0,8.5,23.8,76.0,518792,Jason Heyward,L,4,0,2016015434,3,True,False,True,False,201600206,sln,chn,Busch Stadium,bottom right,none,none
5,593372,Carlos Martinez,R,0,1,1,2,Four-Seamer,"In Play, Out","In Play, Out",Flyout,91.6,84.9,-3.110000,11.290000,2331.577,1.190000,2.570000,19.3,3.0,23.8,46.0,518792,Jason Heyward,L,4,0,2016015434,3,True,False,True,False,201600206,sln,chn,Busch Stadium,right,Slider,bottom right
6,593372,Carlos Martinez,R,0,0,1,1,Four-Seamer,Swinging Strike,Swinging Strike,Flyout,90.0,83.4,4.650000,8.730000,1937.552,-0.440000,4.190000,-20.7,4.1,23.8,50.0,518792,Jason Heyward,L,4,0,2016015451,5,True,False,False,False,201600206,sln,chn,Busch Stadium,high,none,none
7,593372,Carlos Martinez,R,0,1,1,2,Four-Seamer,Ball,Ball,Flyout,90.7,84.1,5.230000,10.370000,2288.122,0.620000,1.490000,-27.8,3.8,23.8,60.0,518792,Jason Heyward,L,4,0,2016015451,5,True,False,False,False,201600206,sln,chn,Busch Stadium,low,Four-Seamer,high
8,593372,Carlos Martinez,R,1,1,1,3,Four-Seamer,Swinging Strike,Foul Ball,Flyout,89.4,83.3,5.540000,9.230000,2106.087,-0.240000,3.590000,-26.2,4.2,23.9,46.0,518792,Jason Heyward,L,4,0,2016015451,5,True,False,False,False,201600206,sln,chn,Busch Stadium,top center,Four-Seamer,low
9,593372,Carlos Martinez,R,1,2,1,4,Curveball,"In Play, Out","In Play, No Out",Flyout,73.7,68.5,-6.200000,-5.210000,1269.201,-0.030000,1.260000,11.6,13.5,23.8,28.0,518792,Jason Heyward,L,4,0,2016015451,5,True,False,False,False,201600206,sln,chn,Busch Stadium,low,Four-Seamer,top center


In [3]:
cards_df = full_df[['pitcher_name', 'count_balls', 'count_strikes', 'outs', \
                   'pitch_number', 'pitch_type', 'batter_stance', 'score_pitcher', 'score_batter', \
                   'inning', 'pitch_location_category', \
                    'previous_pitch_type', 'previous_pitch_location']]
cards_df.head()

,pitcher_name,count_balls,count_strikes,outs,pitch_number,pitch_type,batter_stance,score_pitcher,score_batter,inning,pitch_location_category,previous_pitch_type,previous_pitch_location
0,Carlos Martinez,0,0,1,1,Two-Seamer,L,0,0,1,left,none,none
1,Carlos Martinez,1,0,1,2,Four-Seamer,L,0,0,1,low,Two-Seamer,left
2,Carlos Martinez,2,0,1,3,Four-Seamer,L,0,0,1,low and right,Four-Seamer,low
3,Carlos Martinez,3,0,1,4,Four-Seamer,L,0,0,1,middle right,Four-Seamer,low and right
4,Carlos Martinez,0,0,1,1,Slider,L,4,0,3,bottom right,none,none


In [4]:
cards_df[['pitcher_name', 'count_balls', 'count_strikes', 'outs', \
                   'pitch_number', 'pitch_type', 'batter_stance', 'score_pitcher', 'score_batter', \
                   'inning', 'pitch_location_category', \
                   'previous_pitch_type', 'previous_pitch_location']] = \
cards_df[['pitcher_name', 'count_balls', 'count_strikes', 'outs', \
                   'pitch_number', 'pitch_type', 'batter_stance', 'score_pitcher', 'score_batter', \
                   'inning', 'pitch_location_category', \
                   'previous_pitch_type', 'previous_pitch_location']].astype(str)

C:\Users\bobvo\Anaconda3\lib\site-packages\pandas\core\frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [5]:
cards_df = cards_df[cards_df.pitch_type != 'Unknown']
cards_df = cards_df[cards_df.pitch_type != 'nan']
cards_df = cards_df[cards_df.pitch_location_category != 'nan']
cards_df.head()

,pitcher_name,count_balls,count_strikes,outs,pitch_number,pitch_type,batter_stance,score_pitcher,score_batter,inning,pitch_location_category,previous_pitch_type,previous_pitch_location
0,Carlos Martinez,0,0,1,1,Two-Seamer,L,0,0,1,left,none,none
1,Carlos Martinez,1,0,1,2,Four-Seamer,L,0,0,1,low,Two-Seamer,left
2,Carlos Martinez,2,0,1,3,Four-Seamer,L,0,0,1,low and right,Four-Seamer,low
3,Carlos Martinez,3,0,1,4,Four-Seamer,L,0,0,1,middle right,Four-Seamer,low and right
4,Carlos Martinez,0,0,1,1,Slider,L,4,0,3,bottom right,none,none


In [6]:
from sklearn.preprocessing import LabelEncoder

name = LabelEncoder()
ptype = LabelEncoder()
stance = LabelEncoder()
plc = LabelEncoder()
ppt = LabelEncoder()
ppl = LabelEncoder()

cards_df["pitcher_name"] = name.fit_transform(cards_df["pitcher_name"])
cards_df["pitch_type"] = ptype.fit_transform(cards_df["pitch_type"])
cards_df["batter_stance"] = stance.fit_transform(cards_df["batter_stance"])
cards_df["pitch_location_category"] = plc.fit_transform(cards_df["pitch_location_category"])
cards_df["previous_pitch_type"] = ppt.fit_transform(cards_df["previous_pitch_type"])
cards_df["previous_pitch_location"] = ppl.fit_transform(cards_df["previous_pitch_location"])
cards_df.head()

,pitcher_name,count_balls,count_strikes,outs,pitch_number,pitch_type,batter_stance,score_pitcher,score_batter,inning,pitch_location_category,previous_pitch_type,previous_pitch_location
0,1,0,0,1,1,12,0,0,0,1,6,15,14
1,1,1,0,1,2,4,0,0,0,1,7,13,7
2,1,2,0,1,3,4,0,0,0,1,9,5,8
3,1,3,0,1,4,4,0,0,0,1,12,5,10
4,1,0,0,1,1,10,0,4,0,3,2,15,14


In [7]:
from keras.utils import to_categorical
encoded_y = cards_df
one_hot_y = to_categorical(encoded_y)
one_hot_y

C:\Users\bobvo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


array([[[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [8]:
target = cards_df["pitch_location_category"]
target_names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']

In [9]:
cols = cards_df[["pitch_location_category"]]

data = cards_df.drop(cols, axis=1)
feature_names = data.columns
data.head()

,pitcher_name,count_balls,count_strikes,outs,pitch_number,pitch_type,batter_stance,score_pitcher,score_batter,inning,previous_pitch_type,previous_pitch_location
0,1,0,0,1,1,12,0,0,0,1,15,14
1,1,1,0,1,2,4,0,0,0,1,13,7
2,1,2,0,1,3,4,0,0,0,1,5,8
3,1,3,0,1,4,4,0,0,0,1,5,10
4,1,0,0,1,1,10,0,4,0,3,15,14


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
train_scores = []
test_scores = []
for k in range(1, 4, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

k: 1, Train/Test Score: 0.949/0.100
k: 3, Train/Test Score: 0.425/0.104


In [12]:
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)
new = np.array([[1,1,1,0,3,1,0,0,1,5,5,5]])

encoded_prediction = knn.predict(new)
encoded_prediction

array([7], dtype=int64)

In [13]:
predicted_pitch_location = plc.inverse_transform(encoded_prediction)
predicted_pitch_location

C:\Users\bobvo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['low'], dtype=object)

In [14]:
print(predicted_pitch_location)

['low']


In [15]:
filename = 'models/pitch_location_model.sav'
pickle.dump(knn, open(filename, 'wb'))